<div class="notebook-buttons" style="display:flex; padding-top: 5rem;padding-bottom: 2.5rem;line-height: 2.15;">
    <a href="https://colab.research.google.com/github/zazuko/notebooks/blob/master/notebooks/animal-disease/epidemics.ipynb">
        <div id="colab-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; align-items: center;">
            <img class="call-to-action-img" src="img//colab.svg" width="30" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Run in Google Colab</div>
        </div>
    </a>
    <a href="https://raw.githubusercontent.com/zazuko/notebooks/master/notebooks/animal-disease/epidemics.ipynb" download>
        <div id="download-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img//download.svg" width="22" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Download Notebook</div>
        </div>
    </a>
    <a href="https://github.com/zazuko/notebooks/blob/master/notebooks/animal-disease/epidemics.ipynb">
        <div id="github-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img//github.svg" width="25" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">View on GitHub</div>
        </div>
    </a>
</div>

# Animal epidemics in Switzerland

FOAG, Federal Office for Agriculture, collects data on the animal diseases in Switzerland. This data is published as [Linked Data](https://en.wikipedia.org/wiki/Linked_data). 

In this tutorial, we will show **how to work with Linked Data.** Mainly, we will see how to work with data on animal epidemics.   
We will look into how to query, process, and visualize it.   

## Ideas

* Animal hierarchy
* Disease hierarchy
* Number of reports per gde, over time (https://python-visualization.github.io/folium/plugins.html#folium.plugins.TimeSliderChoropleth)

* Serious outbreaks: find reports where stock == killed

In [ ]:
import json

import folium
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from graphly.api_client import SparqlClient


import networkx as nx

from bokeh.io import output_file, show, output_notebook
from bokeh.plotting import figure, from_networkx, ColumnDataSource

from bokeh.models import (Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,CustomJS)
from bokeh.palettes import Spectral4
from bokeh.models import Legend, LegendItem

import pandas as pd

In [ ]:
sparql = SparqlClient("https://int.lindas.admin.ch/query")
geosparql = SparqlClient("https://ld.geo.admin.ch/query")

sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "cube": "<https://cube.link/>",
    "admin": "<https://schema.ld.admin.ch/>",
    "skos": "<http://www.w3.org/2004/02/skos/core#>",
    "disease": "<https://environment.ld.admin.ch/foen/animal-pest/>"   
})

geosparql.add_prefixes({
    "dct": "<http://purl.org/dc/terms/>",
    "geonames": "<http://www.geonames.org/ontology#>",
    "schema": "<http://schema.org/>",
    "geosparql": "<http://www.opengis.net/ont/geosparql#>",
})

### Animals hierarchy

In [ ]:
query = """
SELECT DISTINCT ?specie ?group
WHERE {
  ?s disease:animal-specie ?specieIRI.
  ?specieIRI schema:name ?specie.
  
  ?specieIRI skos:broader/schema:name ?group.
  
  FILTER (LANG(?specie) = "de")
  FILTER (LANG(?group) = "de")
} 
ORDER BY ?group
"""

df = sparql.send_query(query)
df.head()

## Disease hierarchy

In [ ]:
query = """
SELECT DISTINCT ?epidemics ?group
WHERE {
  ?s disease:epidemics ?epidemicsIRI.
  ?epidemicsIRI schema:name ?epidemics.
  
  ?epidemicsIRI skos:broader/schema:name ?group.

  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?group) = "de")
}
ORDER BY ?group
"""

df = sparql.send_query(query)
df.head()

## Can we link disease to animal type?

In [ ]:
query = """
SELECT DISTINCT ?epidemics ?specie ?group
WHERE {
  <https://environment.ld.admin.ch/foen/animal-pest/observation/> cube:observation ?obs .
  ?obs disease:epidemics/schema:name ?epidemics;
       disease:animal-specie ?specieIRI.
       
  ?specieIRI schema:name ?specie.
  ?specieIRI skos:broader/schema:name ?group.
  
  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?specie) = "de")
  FILTER (LANG(?group) = "de")
} 

ORDER BY ?specie
"""

df = sparql.send_query(query)
df.head()

In [ ]:
g = nx.from_pandas_edgelist(df, source='epidemics', target='specie')
groups = {key: "epidemics" for key in df.epidemics} | {key: "specie" for key in df.specie}

nx.set_node_attributes(g, groups, name="group")

colormap = {"epidemics": "yellow", "specie": "green"}
colors = {k: colormap[v] for k, v in groups.items()}
nx.set_node_attributes(g, colors, name="color")

In [ ]:
# Loayouts https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html
plot = Plot()
plot.title.text = 'Bokeh Plot'

graph_renderer = from_networkx(g, nx.kamada_kawai_layout, scale = 1, center = (0,0))

# manipulating nodes
graph_renderer.node_renderer.glyph = Circle(size = 15, fill_color = "color")
graph_renderer.node_renderer.selection_glyph = Circle(size = 15, fill_color = 
"color")
graph_renderer.node_renderer.hover_glyph = Circle(size = 15, fill_color = 
 Spectral4[1])
graph_renderer.node_renderer.data_source.data['group'] = [groups[g] for g in graph_renderer.node_renderer.data_source.data["index"]]

# manipulating edges
graph_renderer.edge_renderer.glyph = MultiLine(line_color = '#CCCCCC', line_alpha = 
 .5, line_width = 5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color = Spectral4[2], 
 line_width = 5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color = Spectral4[1], 
 line_width = 5)


graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph_renderer)

code = '''
    if (cb_data.index.indices.length > 0) {
        const index = cb_data.index.indices[0];

        if (source.data.group[index] === "specie") {
            hover.tooltips = [["Animal", "@index"]];  
        }
        else {
            hover.tooltips = [["Disease", "@index"]];
        }                                     
    }
'''

hover = HoverTool(
    renderers=[graph_renderer.node_renderer]
)
hover.callback = CustomJS(
    args = dict(source = graph_renderer.node_renderer.data_source, hover = hover),
    code = code
)

plot.add_tools(hover, TapTool())

output_notebook()
show(plot)

### Diseases
All reports. Run in browser: https://s.zazuko.com/24fSKE

In [ ]:
query = """
SELECT ?diagnosis ?gde ?specie ?stock ?sick ?infected ?killed ?deceased ?epidemics ?death_cause
WHERE {
  <https://environment.ld.admin.ch/foen/animal-pest/observation/> cube:observation ?obs .
  ?obs disease:epidemics/schema:name ?epidemics;
       disease:diagnosis-date ?diagnosis;
       disease:animals-stock ?stock;
       disease:animals-sick ?sick;
       disease:animals-infected ?infected;
       disease:animals-killed ?killed;
       disease:animals-deceased ?deceased;
       disease:internet-publication ?date;
       disease:death-cause/schema:name ?death_cause;
       disease:animal-specie/schema:name ?specie;
       schema:containedInPlace/schema:name ?gde .
  
  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?death_cause) = "de")
  FILTER (LANG(?specie) = "de")
} 
ORDER BY DESC(?diagnosis) ?gde
"""
df = sparql.send_query(query)
df.head()

### Deadly epidemics

Which epidemics lead to death of all animals?

In [ ]:
query = """
SELECT ?diagnosis ?gde ?specie ?stock_deceased ?epidemics ?death_cause
WHERE {
  <https://environment.ld.admin.ch/foen/animal-pest/observation/> cube:observation ?obs .
  ?obs disease:epidemics/schema:name ?epidemics;
       disease:diagnosis-date ?diagnosis;
       disease:animals-stock ?stock_deceased;
       disease:animals-deceased ?stock_deceased;
       disease:internet-publication ?date;
       disease:death-cause/schema:name ?death_cause;
       disease:animal-specie/schema:name ?specie;
       schema:containedInPlace/schema:name ?gde .
  
  FILTER (?stock_deceased > 1)
  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?death_cause) = "de")
  FILTER (LANG(?specie) = "de")
} 
ORDER BY DESC(?stock_deceased)
"""

df = sparql.send_query(query)
df.head()

A disease can be higly contagious. It may be hence sefer to kill all animal stock at certain farm, and prevent potential disease spread.

Which epidemics forced farmers to kill all their stock?

In [ ]:
query = """
SELECT ?diagnosis ?gde ?specie ?sick ?infected ?deceased ?stock_killed ?epidemics ?death_cause
WHERE {
  <https://environment.ld.admin.ch/foen/animal-pest/observation/> cube:observation ?obs .
  ?obs disease:epidemics/schema:name ?epidemics;
       disease:diagnosis-date ?diagnosis;
       disease:animals-stock ?stock_killed;
       disease:animals-sick ?sick;
       disease:animals-infected ?infected;
       disease:animals-killed ?stock_killed;
       disease:animals-deceased ?deceased;
       disease:internet-publication ?date;
       disease:death-cause/schema:name ?death_cause;
       disease:animal-specie/schema:name ?specie;
       schema:containedInPlace/schema:name ?gde .
  
  FILTER (?stock_killed > 1)
  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?death_cause) = "de")
  FILTER (LANG(?specie) = "de")
} 
ORDER BY DESC(?stock_killed)
"""

df = sparql.send_query(query)
df.head()